# Utilities

Useful helpers

`TinychatAgent.create` gives you a new agent with tinychat lexicon namespace
attached. If you don't pass an instance of atproto agent into create, it will
try to create and authenticate a test agent using `TEST_AGENT_SERVICE`,
`TEST_AGENT_IDENTIFIER`, `TEST_AGENT_PASSWORD` env vars. For example,

```
TEST_AGENT_SERVICE=https://bsky.social
TEST_AGENT_IDENTIFIER=foo
TEST_AGENT_PASSWORD=bar
```

In [ ]:
//| export

import { Agent, AtpAgent } from "@atproto/api";
import { ChatNS } from "tinychat/api/index.ts";

export class TinychatAgent {
  public chat: ChatNS;
  public agent: Agent;

  constructor(agent: Agent) {
    this.chat = new ChatNS(agent);
    this.agent = agent;
  }

  static async create(agent?: Agent | undefined): Promise<TinychatAgent> {
    if (agent) {
      return new TinychatAgent(agent);
    }

    const [service, identifier, password] = [
      Deno.env.get("TEST_AGENT_SERVICE"),
      Deno.env.get("TEST_AGENT_IDENTIFIER"),
      Deno.env.get("TEST_AGENT_PASSWORD"),
    ];

    if (!service || !identifier || !password) {
      throw new Error(
        "Missing TEST_AGENT_SERVICE, TEST_AGENT_IDENTIFIER, or TEST_AGENT_PASSWORD",
      );
    }

    const testAgent = new AtpAgent({ service });
    await testAgent.login({ identifier, password });

    return new TinychatAgent(testAgent);
  }
}

In [ ]:
import { assert } from "asserts";

Deno.test("make sure test client works", async () => {
  const ta = await TinychatAgent.create();
  const { records } = await ta.chat.tinychat.server.list({
    repo: ta.agent.assertDid,
  });
  assert(typeof records.length !== "undefined");
});